In [2]:
from preprocess import dataset,get_Kfold,fit_transform,transform
from sklearn.preprocessing import MinMaxScaler
from models import FM_MLP
from torch import optim
from torch.utils.data import DataLoader,Dataset
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd

def same_seeds(seed1,seed2,seed3,seed4,seed5):
    random.seed(seed1)
    np.random.seed(seed2)
    torch.manual_seed(seed3)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed4)
        torch.cuda.manual_seed_all(seed5)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True



In [3]:
# 随机种子选取
seed1 = random.randint(1,2147483647)
seed2 = random.randint(1,2147483647)
seed3 = random.randint(1,2147483647)
seed4 = random.randint(1,2147483647)
seed5 = random.randint(1,2147483647)

print(seed1,seed2,seed3,seed4,seed5)
same_seeds(seed1,seed2,seed3,seed4,seed5)

311055986 1668586706 876123019 1342868038 398597252


In [4]:
def get_models(model,input_size):
    if model=='FM_MLP':
        return FM_MLP(input_size,64)

In [5]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
file = './data/jiji small126.xlsx'
para_path = './save/parameter'
batch_size = 300
plot = True

In [6]:
for config in config_list:
    Type = config['Type']
    N_size = config['N_size']
    num_epoch = config['num_epoch']
    lr = config['lr']
    k = config['k']
    alpha,beta = config['alpha'],config['beta']
    input_size = config['input_size']
    
    K_fold_data = get_Kfold(file,k)
    same_seeds(seed1,seed2,seed3,seed4,seed5)
    rmse_list = []
    
    for k_step,(trian_data,test_data) in enumerate(K_fold_data):
        print(f"-----------[ K:{k_step+1} Type:{Type} N_size:{N_size} ]-----------")
        scaler = MinMaxScaler()
        trian_data,scaler = fit_transform(scaler,trian_data,input_size)
        test_data,scaler = transform(scaler,test_data,input_size)
        
        model = get_models(config['model'],input_size).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr=lr)
        lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer,lambda x: 1 - x/num_epoch,last_epoch=-1)
        
        same_seeds(seed1,seed2,seed3,seed4,seed5)
        
        trian_data = dataset(trian_data,Type,N_size,input_size)
        test_data = dataset(test_data,Type,N_size,input_size)
      
        train_loader = DataLoader(dataset=trian_data,batch_size=batch_size)
        test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
        
        train_MLP(train_loader,model,criterion,optimizer,num_epoch,k_step,lr_scheduler,para_path)
        total_mse = test_MLP(test_loader,model,k_step,criterion,plot,para_path)
        rmse_list.append(total_mse*len(K_fold_data[k_step-1][1]))
        

    print(f"total rmse of k fold is: {(sum(rmse_list)/len(data))**0.5}")
    print(f"seed = {seed1},{seed2},{seed3},{seed4},{seed5}")

-----------[ K:1 Type:odd N_size:<126 ]-----------


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 64])